In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os, sys, glob
from tqdm import tqdm
from model import model1, model2
sys.path.insert(0,"/home/msmith/misc/py/")
from performance import Performance
from hStackBatch import hStackBatch
from loadData import loadData
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams["figure.figsize"] = 20, 10

def displayBatch(X,names,predictions):
    try:
        #X,Y = XY
        #names = train.decodeToName(Y)
        X *= 255.0
        bs = X.shape[0]
        X = X.astype(np.uint8)[:,:,:,::-1]
        fig = plt.figure(figsize=(40,20))
        idx = 0
        for i in range(1,bs+1):
            ax = fig.add_subplot(1,bs+1,i)
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            plt.imshow(X[idx])
            if names[idx] == predictions[idx]:
                correct = 1
            else:
                correct = 0
            ax.set_title("Truth = {0}, Pred = {1}, ({2})".format(names[idx],predictions[idx],correct))
            idx +=1
        plt.show()
    except IndexError:
        print("OOB")
        
def decodeLabel(label):
    '''
    Takes whale label eg. label = 125.
    Returns: whale name eg. whale_27860
    '''
    
    whalePath = df.loc[df.label == label].fullPath
    whaleName = whalePath.iloc[0].split("/")[2].replace("whale_","")
    return whaleName
decodeLabel = np.vectorize(decodeLabel)

def varSummary(var):
    with tf.name_scope("summary"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean",var)
        tf.summary.histogram("mean",var)

In [ ]:
if __name__ == "__main__":
    batchSize  = 10
    batchCapacity = 40
    nThreads = 32

    ## HyperParameter defaults
    h, w, c = 500, 500, 3
    nClasses = 447
    inDims = [None,h,w,c]
    nFeatsInit = 32
    nFeatsInc = 32
    #keepProb = 0.9 # dropout
   
    load = 0
    display = 1
    disFreq = 400
    lr = 3e-4
    nEpochs = 5
    trainAll = False
        
    def nodes(trainOrTest):
        assert trainOrTest in ["train","test"], "Please specify either 'train' or 'test."
        if trainOrTest == "train":
            csvPath = "trainCV.csv"
            shuffle = True
            keepProb = 0.5
            is_training=True
        else:
            csvPath = "testCV.csv"
            shuffle = False
            keepProb = 1.0
            is_training = False
        x, y, yPaths = loadData(csvPath,shape=[h,w,c],batchSize=batchSize,batchCapacity=batchCapacity,nThreads=nThreads,shuffle=shuffle)
        # Define placeholders and model
        yPred = model1(x,inDims=inDims,nClasses=nClasses,nFeatsInit=nFeatsInit,nFeatsInc=nFeatsInc,keepProb=keepProb,training=is_training) # model
        ce = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(yPred,y))
        
        correct = tf.equal(tf.argmax(yPred,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        trainStep = None
        if trainOrTest == "train":
            learningRate = tf.placeholder(tf.float32)
            trainStep = tf.train.AdamOptimizer(learningRate).minimize(ce)
            return x, y, yPaths, yPred, ce, correct, accuracy, trainStep, learningRate
        else:
            return x, y, yPaths, yPred, ce, correct, accuracy, _, _
    
    modelName = "models/model2.tf"


In [ ]:
if __name__ == "__main__":
    for epoch in xrange(1):
        for trTe in ["train"]:
            if epoch > 0:
                load = 1 # Must have a model to load by 2nd epoch... so load it !
                tf.reset_default_graph()
            x, y, yPaths, yPred, ce, correct, accuracy, trainStep, learningRate = nodes(trTe)
            [varSummary(x) for x in [ce,accuracy]] # Summaries
            merged = tf.summary.merge_all()
            saver = tf.train.Saver()
            with tf.Session() as sess:
                if load == 1:
                    saver.restore(sess,modelName)
                writer = tf.summary.FileWriter("summary/{0}".format(trTe),sess.graph)
                tf.global_variables_initializer().run()
                tf.local_variables_initializer().run()
                coord = tf.train.Coordinator()
                threads = tf.train.start_queue_runners(sess=sess,coord=coord)
                count = 0
                try:
                    while True:      
                        if trTe == "train":
                            _,summary = sess.run([trainStep,merged],feed_dict={learningRate:lr})
                            writer.add_summary(summary,count)
                            count += 1
                        if coord.should_stop():
                            break
                except Exception, e:
                    coord.request_stop(e)
                finally:
                    coord.request_stop()
                    coord.join(threads)
                saver.save(sess,modelName)
                sess.close()

Shuffling csv
(10, 500, 500, 3)
(10, 250, 250, 32)
(10, 125, 125, 64)
(10, 63, 63, 96)
(10, 32, 32, 128)
(10, 16, 16, 160)
(10, 8, 8, 192)
(10, 4, 4, 224)
(10, 2, 2, 256)
